In [2]:
# Loading the required Libraries and loading the data a dataframe
import pandas as pd
import sqlite3

df = pd.read_excel('Rates_DF_Example.xls')


In [3]:
# Dropping Null Values
df.dropna(inplace=True)

In [4]:
# Dropping Duplicate Values
df.drop_duplicates(inplace=True)

In [5]:
# Reading the for undestanding of various columns
df

,service_code,service_code_description,hospital_ID,cost
0,29870,Knee surgery,1003820630,1432.34
1,29870,Knee surgery,1003820630,1016.51
2,29850,Hip Surgery,1003820630,1519.18
4,22818,Covid Vaccine,1003820630,5402.73
6,29870,Knee surgery,1013970078,1432.34
...,...,...,...,...
10747,29850,Hip Surgery,95-4565259,1519.18
10749,22818,Covid Vaccine,95-4565259,5402.73
10751,29870,Knee surgery,95-4662001,622.72
10752,29850,Hip Surgery,95-4662001,664.62


In [6]:
# Getting the information about the various types of columns in the database
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6159 entries, 0 to 10753
Data columns (total 4 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   service_code              6159 non-null   int64  
 1   service_code_description  6159 non-null   object 
 2   hospital_ID               6159 non-null   object 
 3   cost                      6159 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 240.6+ KB


In [7]:
# Creating a function as given int he assignment to calulate the costliest and cheapeast hostipals according to the service provided.
def calc_hospital_rankings(df):
    res = []

    unique_codes = df['service_code'].unique()
    
    for code in unique_codes:
        sub = df[df['service_code'] == code]
        unique_desc = sub['service_code_description'].unique()
        
        for desc in unique_desc:
            desc_sub = sub[sub['service_code_description'] == desc]
            max_cost_hosp = desc_sub['hospital_ID'][desc_sub['cost'].idxmax()]
            min_cost_hosp = desc_sub['hospital_ID'][desc_sub['cost'].idxmin()]
            
            res.append({
                'service_code': code,
                'service_code_description': desc,
                'costliest_hospital': max_cost_hosp,
                'cheapest_hospital': min_cost_hosp
            })
    
    return pd.DataFrame(res)
hospital_rankings = calc_hospital_rankings(df)


In [8]:
hospital_rankings

,service_code,service_code_description,costliest_hospital,cheapest_hospital
0,29870,Knee surgery,20-8624691,20-8075502
1,29850,Hip Surgery,20-8624691,20-8075502
2,22818,Covid Vaccine,20-8624691,27-1145142


In [9]:
# Creating as well as connecting to the SQL lite database and loading the dataframe to the sqlite database.
db_conn = sqlite3.connect('hospital_data.db')
df.to_sql('hospital_data', db_conn, if_exists='replace', index=False)

6159

In [10]:
# Creating the query in accordance to the assignment for printing the mean and standard devation for cost for the various services.
query = '''
SELECT
    h.service_code,
    AVG(h.cost) AS mean_cost,
    SQRT(AVG((h.cost - s.avg_cost) * (h.cost - s.avg_cost))) AS sample_std_deviation
FROM
    hospital_data AS h
JOIN (
    SELECT
        service_code,
        AVG(cost) AS avg_cost
    FROM
        hospital_data
    GROUP BY
        service_code
) AS s
ON h.service_code = s.service_code
GROUP BY
    h.service_code;

'''

mean_std_result = pd.read_sql_query(query, db_conn)
db_conn.close()

In [11]:
mean_std_result

,service_code,mean_cost,sample_std_deviation
0,22818,3402.541164,1709.510988
1,29850,965.211716,513.016666
2,29870,820.209469,468.551561
